In [30]:
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import data_utils
import download
from scipy.stats import skew, kurtosis
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2 
import keras
from keras import layers
from sklearn.metrics import accuracy_score
from keras import ops

In [31]:
def loadData(path):
    listOfTestFiles = os.listdir(path=path)
    train = []
    train_labels = []
    test = []
    test_labels = []
        
        
    print("Training files = ",listOfTestFiles[1:6])
    #For collecting Training data:
    for file in listOfTestFiles[1:6]:
        with open(path+file,'rb') as fo:
            dict = pickle.load(fo,encoding='bytes')
            train.append(dict[b'data'])
            train_labels.append(dict[b'labels'])

    print(listOfTestFiles[7])
    #for collecting Testing data
    with open(path+listOfTestFiles[7],'rb') as fo:
            dict = pickle.load(fo,encoding='bytes')
            test.append(dict[b'data'])
            test_labels.append(dict[b'labels'])

    dictData = {}
    dictData['train_data'] = np.reshape(np.array(train),newshape=(np.array(train).shape[0]*np.array(train).shape[1],np.array(train).shape[2]))
    dictData['train_labels'] = np.reshape(np.array(train_labels),newshape=(np.array(train_labels).shape[0]*np.array(train_labels).shape[1]))
    dictData['test_data'] = np.reshape(np.array(test),newshape=(np.array(test).shape[0]*np.array(test).shape[1],np.array(test).shape[2]))
    dictData['test_labels'] = np.reshape(np.array(test_labels),newshape=(np.array(test_labels).shape[0]*np.array(test_labels).shape[1]))
    return dictData

In [32]:
#inputs will be of shape N X F where N = number of examples and F = number of features for each image.
#labels will be of size N,1. This consists of labels for all N labels from 10 classes

class kNearestNeighbour(object):
    def __init__(self):
        pass
    
    def train(self,X,Y):
        #Knn will remember all of its training data
        self.Xtr = X
        self.Ytr = Y
        
    def predict(self, X, k):
        # Ensure k is not larger than the training dataset
        if k > self.Xtr.shape[0]:
            raise ValueError(f"k={k} is greater than the number of training samples={self.Xtr.shape[0]}")
        
        # Number of test samples
        test_samples = X.shape[0]
        Ypred = np.zeros(test_samples, dtype=self.Ytr.dtype)
        
        for i in range(test_samples):
            print(f"Test example = {i}", end="\r")
            
            # Compute L1 distances
            dist = np.sum(np.abs(X[i, :] - self.Xtr), axis=1)
            
            # Find the indices of the k smallest distances
            idx = np.argpartition(dist, k)[:k]
            
            # Count the labels of the nearest neighbors
            label_count = np.zeros(10, dtype=self.Ytr.dtype)
            for x in idx:
                label_count[int(self.Ytr[x])] += 1
            
            # Assign the label with the maximum count
            Ypred[i] = np.argmax(label_count)
        
        return Ypred


In [33]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

In [34]:
# Normalize the data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Flatten the data
x_train_flat = x_train.reshape(x_train.shape[0], -1)  # (50000, 32*32*3)
x_test_flat = x_test.reshape(x_test.shape[0], -1)    # (10000, 32*32*3)

# Reshape labels to 1D
y_train = y_train.flatten()
y_test = y_test.flatten()

In [ ]:
# Initialize the KNN classifier
knn = kNearestNeighbour()

# Train the classifier
knn.train(x_train_flat, y_train)

# Predict the labels for a subset of test data (e.g., 100 samples)
num_test_samples = 100  # To save time, use a small subset for testing
y_pred = knn.predict(x_test_flat[:num_test_samples], k=5)

# Calculate accuracy
accuracy = accuracy_score(y_test[:num_test_samples], y_pred)
print(f"Accuracy: {accuracy:.2f}%")

Accuracy: 35.00%9
